In [110]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go
df = pd.read_csv('data/opportunities.csv',low_memory=False)
df = df.rename(columns={"Source ": "Source"})
df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None


In [111]:
df.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currency

In [112]:
df_amount = df.loc[:,['Price','Currency','Total_Amount','Total_Amount_Currency','ASP','ASP_Currency','ASP_(converted)','ASP_(converted)_Currency','Total_Taxable_Amount','Total_Taxable_Amount_Currency']]
df_amount.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 10 columns):
Price                            16947 non-null object
Currency                         16947 non-null object
Total_Amount                     16888 non-null float64
Total_Amount_Currency            16947 non-null object
ASP                              13738 non-null float64
ASP_Currency                     16947 non-null object
ASP_(converted)                  13738 non-null float64
ASP_(converted)_Currency         16947 non-null object
Total_Taxable_Amount             16947 non-null float64
Total_Taxable_Amount_Currency    16947 non-null object
dtypes: float64(4), object(6)
memory usage: 1.3+ MB


In [113]:
df_amount["Price"] = pd.to_numeric(df_amount["Price"],errors='coerce')
df_amount["Total_Amount"] = pd.to_numeric(df_amount["Total_Amount"],errors='coerce')
df_amount["ASP"] = pd.to_numeric(df_amount["ASP"],errors='coerce')
df_amount["ASP_(converted)"] = pd.to_numeric(df_amount["ASP_(converted)"],errors='coerce')
df_amount["Total_Taxable_Amount"] = pd.to_numeric(df_amount["Total_Taxable_Amount"],errors='coerce')
df_amount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 10 columns):
Price                            356 non-null float64
Currency                         16947 non-null object
Total_Amount                     16888 non-null float64
Total_Amount_Currency            16947 non-null object
ASP                              13738 non-null float64
ASP_Currency                     16947 non-null object
ASP_(converted)                  13738 non-null float64
ASP_(converted)_Currency         16947 non-null object
Total_Taxable_Amount             16947 non-null float64
Total_Taxable_Amount_Currency    16947 non-null object
dtypes: float64(5), object(5)
memory usage: 1.3+ MB


In [114]:
df_show_amount = df_amount.count().to_frame().reset_index()
df_show_amount.columns = ["Columna","Cantidad de Registros con datos validos"]
df_show_amount

,Columna,Cantidad de Registros con datos validos
0,Price,356
1,Currency,16947
2,Total_Amount,16888
3,Total_Amount_Currency,16947
4,ASP,13738
5,ASP_Currency,16947
6,ASP_(converted),13738
7,ASP_(converted)_Currency,16947
8,Total_Taxable_Amount,16947
9,Total_Taxable_Amount_Currency,16947


In [115]:
colorscale = [[0, '#91860d'],[.5, '#d6cc5a'],[1, '#f5efb0']]
table = ff.create_table(df_show_amount,colorscale=colorscale)
py.iplot(table)

In [116]:
df_geografic = df.loc[:,["Region","Territory","Billing_Country"]]
df_geografic["Territory"] = df_geografic["Territory"].replace("None",np.NaN)
df_geografic["Region"] = df_geografic["Region"].replace("None",np.NaN)
df_geografic["Billing_Country"] = df_geografic["Billing_Country"].replace("None",np.NaN)
df_geografic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 3 columns):
Region             16947 non-null object
Territory          11948 non-null object
Billing_Country    16920 non-null object
dtypes: object(3)
memory usage: 397.3+ KB


In [117]:
df_geografic_show = df_geografic.count().to_frame().reset_index()
df_geografic_show.columns = ["Columna","Cantidad de Registros con datos validos"]
df_geografic_show

,Columna,Cantidad de Registros con datos validos
0,Region,16947
1,Territory,11948
2,Billing_Country,16920


In [118]:
colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
table = ff.create_table(df_geografic_show,colorscale=colorscale)
py.iplot(table)

In [119]:
df_categoric = df.loc[:,["Source","Opportunity_Type","Quote_Type","Delivery_Terms","Brand","Product_Type","Size",\
                         "Product_Family","Product_Name","Prod_Category_A","Product_Category_B","Stage"]]

df_categoric["Source"] = df_categoric["Source"].replace("None",np.NaN)
df_categoric["Opportunity_Type"] = df_categoric["Opportunity_Type"].replace("None",np.NaN)
df_categoric["Quote_Type"] = df_categoric["Quote_Type"].replace("None",np.NaN)
df_categoric["Delivery_Terms"] = df_categoric["Delivery_Terms"].replace("None",np.NaN)
df_categoric["Brand"] = df_categoric["Brand"].replace("None",np.NaN)
df_categoric["Product_Type"] = df_categoric["Product_Type"].replace("None",np.NaN)
df_categoric["Size"] = df_categoric["Size"].replace("None",np.NaN)
df_categoric["Product_Family"] = df_categoric["Product_Family"].replace("None",np.NaN)
df_categoric["Product_Name"] = df_categoric["Product_Name"].replace("None",np.NaN)
df_categoric["Prod_Category_A"] = df_categoric["Prod_Category_A"].replace("None",np.NaN)
df_categoric["Product_Category_B"] = df_categoric["Product_Category_B"].replace("None",np.NaN)
df_categoric["Stage"] = df_categoric["Stage"].replace("None",np.NaN)
df_categoric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 12 columns):
Source                7450 non-null object
Opportunity_Type      16947 non-null object
Quote_Type            16947 non-null object
Delivery_Terms        16947 non-null object
Brand                 1036 non-null object
Product_Type          1012 non-null object
Size                  980 non-null object
Product_Family        16947 non-null object
Product_Name          16947 non-null object
Prod_Category_A       16947 non-null object
Product_Category_B    1019 non-null object
Stage                 16947 non-null object
dtypes: object(12)
memory usage: 1.6+ MB


In [120]:
df_categoric_show = df_categoric.count().to_frame().reset_index()
df_categoric_show.columns = ["Columna","Cantidad de Registros con datos validos"]
df_categoric_show

,Columna,Cantidad de Registros con datos validos
0,Source,7450
1,Opportunity_Type,16947
2,Quote_Type,16947
3,Delivery_Terms,16947
4,Brand,1036
5,Product_Type,1012
6,Size,980
7,Product_Family,16947
8,Product_Name,16947
9,Prod_Category_A,16947


In [121]:
colorscale = [[0, '#0c8c06'],[.5, '#65d95f'],[1, '#d7f5d5']]
table = ff.create_table(df_categoric_show,colorscale=colorscale)
py.iplot(table)